In [ ]:
import numpy as np
import pandas as pd

#Import Data


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# path = path to raw essays
raw_essays = pd.read_csv(path, index_col=0)

# Data Augmentation with *ChatGPT*

In [ ]:
# path_to_already_augmented_IDs = path to a file that stores the IDs of essays that have already been augmented
# path_to_augmented_essays = path to a file that contains the augmented essays
augmented_IDS = open(path_to_already_augmented_IDs).read().splitlines()

In [ ]:
!pip install tiktoken
!pip install cohere
!pip install openai

In [ ]:
downgrading_prompt='I am building a model to automatically grade Arabic essays according to CEFR. I need more data and I need you to help me with data augmentation.'+ 'I will give you an Arabic essay that was given the CEFR grade of {}.'+'I want you to modify it so it scores {}. You should simplify grammatically complex sentences'+'and replace {}-level words with {}-level ones. You can also introduce some spelling errors, at the rate expected of the target level. Here is the essay:\' {} \' '



In [ ]:
upgrading_prompt = 'I am building a model to automatically grade Arabic essays according to CEFR. I need more data and I need you to help me with data augmentation.'+'I will give you an Arabic essay that was given the CEFR grade of {}.'+'I want you to modify it so it scores {}. You should fix some or all errors and make it more grammatically complex to suit the target level.'+'You should also replace {}-level words with {}-level ones. Here is the essay:\' {} \' '

In [ ]:
import requests

headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer <api=key>',
}


for ID in raw_essays.index[110:120]:
  if ID not in augmented_IDS:
    augmented_df = pd.DataFrame(columns=['Document', 'Raw', 'to_grade', 'from_grade'])
    ## downgrade:
    from_grade = raw_essays.loc[ID]['grade']
    if(from_grade in ['C1', 'B2', 'B1']):
      for to_grade in ['A1', 'A2']:
        json_data = {
          'model': 'gpt-4',
          'messages': [
          {
              'role': 'user',
              'content' :  downgrading_prompt.format(from_grade, to_grade, from_grade, to_grade, raw_essays.loc[ID]['Raw']),},
              ],
            'temperature': 0.5,
            }
        response = requests.post('https://api.openai.com/v1/chat/completions', headers=headers, json=json_data)
        augmented_df = augmented_df.append({'Document': ID, 'Raw': response.json()['choices'][0]['message']['content'], 'to_grade': to_grade, 'from_grade': from_grade}, ignore_index =True)

    ## upgrade:
    if(from_grade in ['A2', 'B1', 'B2']):
      for to_grade in ['C1', 'C2']:
        json_data = {
          'model': 'gpt-4',
          'messages': [
          {
            'role': 'user',
            'content' :  upgrading_prompt.format(from_grade, to_grade, from_grade, to_grade, raw_essays.loc[ID]['Raw']),},
            ],
            'temperature': 0.5,
            }
        response = requests.post('https://api.openai.com/v1/chat/completions', headers=headers, json=json_data)
        augmented_df = augmented_df.append({'Document': ID, 'Raw': response.json()['choices'][0]['message']['content'], 'to_grade': to_grade, 'from_grade': from_grade}, ignore_index =True)


    #attach augmented essays to augmented_augmented.tsv file:
    with open(path_to_augmented_essays, 'a') as f:
      augmented_df.to_csv(f, sep='\t', index=False, header = False)


    ##attach ID to /content/drive/MyDrive/NLP701/Augmented_Essays/augmented_ids.txt:
    augmented_IDS.append(ID)
    with open(path_to_already_augmented_IDs, 'a') as f:
      f.write(ID + '\n')

